In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *
from lesview.diags import get_power_spectrum_1d

In [ ]:
g = 9.81
H = 30
u10 = 8
N2 = 1.962e-4
bstar = N2 * H
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)
amplitude = 1.0
wavelength = 60
wavenumber = 2.*np.pi/wavelength
frequency = np.sqrt(g*wavenumber*np.tanh(wavenumber*H))
us0 = amplitude**2*wavenumber
la = np.sqrt(ustar/us0)
print(la)

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2_iles9'
datapath = os.path.join(os.path.pardir, 'oceananigans', '{:s}'.format(casename))
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
filepath = os.path.join(datapath, 'fields.jld2')
data_fld = OceananigansDataVolume(filepath=filepath, fieldname=['b','w'])

In [ ]:
ds = data_fld.dataset
ds

In [ ]:
w = ds.data_vars['w'].isel(time=30).sel(zi=-17, method='nearest')
w.plot()

In [ ]:
k, a = get_power_spectrum_1d(w)
plt.plot(k,a)
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
labels = ['w']
L = (ds.x[0]+ds.x[-1]).values
nx = ds.x.size
fig = plt.figure(figsize=[6,4])
for i, vel in enumerate([ w]):
    e_x, e_y = get_power_spectrum_1d(vel)
    e_x *= 2.*np.pi/L
    plt.plot(e_x, e_y, label=labels[i])
plt.xscale('log')
plt.yscale('log')
xx = e_x[1:]
yy = xx**(-5/3)*1e3
plt.plot(xx, yy, color='gray')
plt.legend(loc='lower center')
# ymax = np.percentile(e_y, 90)*1e2
# ymin = np.percentile(e_y, 10)*1e-1
ymax = 10**5
ymin = 10**(-1)
plt.ylim([ymin, ymax])
plt.vlines(2*np.pi/L, ymin=ymin, ymax=ymax, color='lightgray')
plt.vlines(2*np.pi/(2*np.sqrt(2)*L/nx), ymin=ymin, ymax=ymax, color='lightgray')
plt.xlabel('$k$')
plt.ylabel('$S(k)$')
# figname = 'PS_{:s}_{:s}'.format(casename, key)
# fig.savefig(figname, dpi=300, facecolor='w')

In [ ]:
ds.data_vars['b'].isel(time=30).sel(z=-20,method='nearest').plot()